In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# income 데이터 로드
file_path = 'income.csv'
income_data = pd.read_csv(file_path)

# GRDP 데이터 로드
grdp_path = 'GRDP.csv'
grdp_data = pd.read_csv(grdp_path)

# 지역이 행, 년도가 컬럼인 데이터 구조에서 'year'과 'region'이 명시적인 컬럼으로 있는 형태로 변환
grdp_data = grdp_data.melt(id_vars=['region'], var_name='year', value_name='GRDP')

# 데이터 타입 변환 (년도를 숫자형으로 변환)
grdp_data['year'] = grdp_data['year'].astype(int)


# 년도와 지역을 기준으로 병합
data = pd.merge(income_data, grdp_data, on=['year', 'region'], how='left')


# 나이 계산 및 비현실적 나이 값 제거
data['age'] = data['year'] - data['year_born']
data = data[(data['age'] >= 0) & (data['age'] <= 120)]

# 음수 수입 값 처리
data['income'] = data['income'].apply(lambda x: x if x >= 0 else None)

# 필수 컬럼에서 NaN 값 제거
data.dropna(subset=['region', 'income', 'family_member', 'gender', 'education_level', 'marriage', 'religion', 'age', 'company_size','GRDP'], inplace=True)

# 교육 수준 라벨링
education_mapping = {
    'No education (under 7 yrs-old)': 1,
    'No education (7 & over 7 yrs-old)': 2,
    'Elementary': 3,
    'Middle school': 4,
    'High school': 5,
    'College': 6,
    'University degree': 7,
    'MA': 8,
    'Doctoral degree': 9
}
data['education_level'] = data['education_level'].map(education_mapping)


# 'company_size' 컬럼에서 NaN이나 빈 문자열을 '0'으로 처리
data['company_size'] = data['company_size'].replace(['', ' '], 0)
data['company_size'].fillna(0, inplace=True)

# 'company_size' 컬럼의 데이터 타입을 정수형으로 변환
data['company_size'] = data['company_size'].astype(int)


# 지역 이동 확인 함수 정의 및 적용 (원-핫 인코딩 전에 적용)
def check_region_movement(df):
    df.sort_values('year', inplace=True)
    df['previous_region'] = df['region'].shift(1)
    # 이전 지역이 NaN인 경우 현재 지역으로 설정
    df['previous_region'] = df['previous_region'].fillna(df['region'])
    df['region_movement'] = df.apply(lambda row: row['region'] if row['region'] != row['previous_region'] else None, axis=1)
    return df

data = data.groupby('id').apply(check_region_movement).reset_index(drop=True)

# 필요없는 컬럼 제거
data.drop(columns=['year_born', 'wave', 'occupation'], inplace=True)

# OneHotEncoder 설정
encoder = OneHotEncoder()
categorical_columns = ['region', 'gender', 'marriage', 'religion']
data_encoded = encoder.fit_transform(data[categorical_columns])
column_names = encoder.get_feature_names_out(categorical_columns)

# 원-핫 인코딩 데이터를 DataFrame으로 변환
data_encoded = pd.DataFrame(data_encoded.toarray(), columns=column_names)

# 인코딩되지 않은 데이터와 병합
data = pd.concat([data.reset_index(drop=True), data_encoded.reset_index(drop=True)], axis=1)
data.drop(columns=categorical_columns, inplace=True)  # 원본 범주형 컬럼 삭제

# 수정된 데이터를 CSV 파일로 저장
output_file_path = 'Processed_Korea_Income_and_Welfare.csv'
data.to_csv(output_file_path, index=False)

print("데이터 처리가 완료되었으며, 다음 파일에 저장되었습니다:", output_file_path)


데이터 처리가 완료되었으며, 다음 파일에 저장되었습니다: Processed_Korea_Income_and_Welfare.csv


In [2]:
#SMOTE사용

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, make_scorer, f1_score
from imblearn.over_sampling import SMOTE
import seaborn as sns




# 수치형 데이터 정규화
scaler = MinMaxScaler()
numeric_columns = ['income', 'family_member', 'age','GRDP']
data[numeric_columns] = scaler.fit_transform(data[numeric_columns])

# 타겟 변수 설정
def categorize_region_movement(row):
    if pd.isna(row['region_movement']):
        return 0  # 떠나지 않음
    elif row['region_Seoul'] == 1 or row['region_Kyeong-gi'] == 1:
        return 1  # 서울, 경기권으로 떠남
    else:
        return 2  # 그 외의 지역으로 떠남

data['region_movement'] = data.apply(categorize_region_movement, axis=1)

data.head()


# 학습에 사용할 피처 선택
feature_columns = numeric_columns +['education_level', 'company_size']+ [col for col in data.columns if col.startswith('region_') and col != 'region_movement' or col.startswith('gender_') or col.startswith('marriage_') or col.startswith('religion_')]
X = data[feature_columns].copy()  # 슬라이스에서 복사
y = data['region_movement']

# 데이터 분할
random_state = 42
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)

# X_train에 타겟 변수 추가
X_train['region_movement'] = y_train

# 상관계수 매트릭스 계산
corr_matrix = X_train.corr()

# 상관계수 매트릭스 시각화
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='coolwarm')
plt.title('Correlation Matrix Between Numerical Features and Target')
plt.show()





In [ ]:
##학습 

# 학습에 사용할 피처 선택
feature_columns = numeric_columns +['education_level', 'company_size']+ [col for col in data.columns if col.startswith('region_') and col != 'region_movement' or col.startswith('gender_') or col.startswith('marriage_') or col.startswith('religion_')]
X = data[feature_columns]
y = data['region_movement']

# 데이터 분할
random_state = 42
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)





# SMOTE 적용
smote = SMOTE(random_state=random_state)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# 모델 학습 및 평가
models = {
    "Logistic Regression": LogisticRegression(random_state=random_state, max_iter=2000, class_weight='balanced'),
    "Gradient Boosting": GradientBoostingClassifier(random_state=random_state),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=random_state, class_weight='balanced'),
    "Decision Tree": DecisionTreeClassifier(random_state=random_state, class_weight='balanced')
}

f1_scorer = make_scorer(f1_score, average='weighted')

for model_name, model in models.items():
    scores = cross_val_score(model, X_train_res, y_train_res, cv=5, scoring=f1_scorer)
    model.fit(X_train_res, y_train_res)
    y_pred = model.predict(X_test)
    print(f"{model_name} Cross-Validation F1 Scores: {scores}")
    print(f"{model_name} Average Cross-Validation F1 Score: {scores.mean()}")
    print(f"{model_name} Classification Report:")
    print(classification_report(y_test, y_pred, zero_division=1))



In [ ]:
# 테스트 데이터 샘플 예측 및 출력
num_samples_per_class = 100  # 각 클래스 별 샘플 수
samples_to_predict = pd.concat([X_test[y_test == i].sample(n=min(num_samples_per_class, len(X_test[y_test == i])), random_state=random_state) for i in y_test.unique()])

true_labels = y_test.loc[samples_to_predict.index]
predictions = {model_name: model.predict(samples_to_predict) for model_name, model in models.items()}

# 예측값과 실제값 비교 출력
for i in samples_to_predict.index:
    print(f"\nSample Index {i}:")
    print(f"True Label: {true_labels.loc[i]}")
    for model_name, preds in predictions.items():
        print(f"{model_name} Prediction: {preds[samples_to_predict.index.get_loc(i)]}")